In [115]:
#AI Trainer for the Adjustments Project


In [1]:
#Load input data
import pandas as pd
import numpy as np

#Load data
df=pd.read_csv('SampleData.csv')
df.head(10)

,AdjustmentType,AdjustmentName,AdjustedField,Drawn,Undrawn,PD,CustomerID,CustomerType,RiskCategory,IsDefaulted,CountryCode,NewVal
0,Amend,GRC_EndMonth,CustomerID,526452,577081,0.76,CUST359147,Retail,Medium,1,DE,CUST450896
1,Amend,Defaulting_Reclassify,RiskCategory,286776,727572,0.64,CUST450896,Retail,High,1,DE,Low
2,Amend,Defaulting_Reclassify,RiskCategory,678402,23975,0.69,CUST238947,SME,Low,0,US,Medium
3,Amend,Market Downturn,IsDefaulted,24728,207714,0.79,CUST665428,Corporate,High,0,US,0.75
4,Amend,Credit_Uplift,RiskCategory,708427,921998,0.97,CUST981372,Corporate,Low,1,US,High
5,Amend,Fix_Broken_Batch,CustomerType,211343,969304,0.39,CUST981372,Retail,High,0,GB,Corporate
6,Amend,Credit_Uplift,RiskCategory,207561,713539,0.84,CUST981372,Retail,Low,1,DE,Low
7,Amend,Credit_Uplift,RiskCategory,805805,699732,0.91,CUST359147,SME,High,0,GB,Low
8,Amend,GRC_EndMonth,CustomerID,410297,388670,0.31,CUST103289,Retail,Medium,0,US,CUST450896
9,Amend,GRC_EndMonth,CustomerType,854928,607952,0.36,CUST981372,SME,Medium,0,US,Retail


In [3]:
#Stage 1: OneHot encode string labels so we can use them in the model

#Build list of all string columns in the dataframe
string_columns=[]
for col in df.columns:
    if df[col].dtype=='object':
        string_columns.append(col)

#Turn output labels into numbers
outputLabels=df["NewVal"].unique()
# #Create a dictionary to map the output labels to numbers
outputLabelsDict={}
for i in range(len(outputLabels)):
    outputLabelsDict[outputLabels[i]]=i
df["NewVal"]=df["NewVal"].map(outputLabelsDict)

df_encoded=pd.get_dummies(df, columns=string_columns)

#Move NewVal to the end of the dataframe
df_encoded=df_encoded[[c for c in df_encoded if c not in ['NewVal']] + ['NewVal']]

df_encoded


,Drawn,Undrawn,PD,IsDefaulted,AdjustmentType_Amend,AdjustmentName_Credit_Uplift,AdjustmentName_Default,AdjustmentName_Defaulting_Reclassify,AdjustmentName_Fix_Broken_Batch,AdjustmentName_GRC_EndMonth,...,CustomerType_Corporate,CustomerType_Retail,CustomerType_SME,RiskCategory_High,RiskCategory_Low,RiskCategory_Medium,CountryCode_DE,CountryCode_GB,CountryCode_US,NewVal
0,526452,577081,0.76,1,1,0,0,0,0,1,...,0,1,0,0,0,1,1,0,0,0
1,286776,727572,0.64,1,1,0,0,1,0,0,...,0,1,0,1,0,0,1,0,0,1
2,678402,23975,0.69,0,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,1,2
3,24728,207714,0.79,0,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,3
4,708427,921998,0.97,1,1,1,0,0,0,0,...,1,0,0,0,1,0,0,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,588110,252794,0.79,0,1,0,0,0,0,1,...,0,1,0,1,0,0,0,1,0,10
996,246542,816935,0.98,1,1,0,0,1,0,0,...,1,0,0,0,0,1,0,0,1,5
997,837374,200917,0.09,1,1,0,0,0,0,1,...,0,0,1,0,1,0,0,0,1,4
998,931512,312982,0.57,1,1,0,0,1,0,0,...,0,1,0,0,1,0,0,1,0,8


In [4]:
#Stage 2: Normalize all numeric values
#Normalize the dataframe

#Normalize Drawn (-1 to 1)
df_normalized=df_encoded.copy()
df_normalized["Drawn"]=(df_encoded["Drawn"]-df_encoded["Drawn"].min())/(df_encoded["Drawn"].max()-df_encoded["Drawn"].min())*2-1
#Normalize Undrawn (-1 to 1)
df_normalized["Undrawn"]=(df_encoded["Undrawn"]-df_encoded["Undrawn"].min())/(df_encoded["Undrawn"].max()-df_encoded["Undrawn"].min())*2-1

#Clip PD to 0-1
df_normalized["PD"]=df_normalized["PD"].clip(0,1)


df_normalized

,Drawn,Undrawn,PD,IsDefaulted,AdjustmentType_Amend,AdjustmentName_Credit_Uplift,AdjustmentName_Default,AdjustmentName_Defaulting_Reclassify,AdjustmentName_Fix_Broken_Batch,AdjustmentName_GRC_EndMonth,...,CustomerType_Corporate,CustomerType_Retail,CustomerType_SME,RiskCategory_High,RiskCategory_Low,RiskCategory_Medium,CountryCode_DE,CountryCode_GB,CountryCode_US,NewVal
0,0.051711,0.154150,0.76,1,1,0,0,0,0,1,...,0,1,0,0,0,1,1,0,0,0
1,-0.428278,0.455191,0.64,1,1,0,0,1,0,0,...,0,1,0,1,0,0,1,0,0,1
2,0.356014,-0.952279,0.69,0,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,1,2
3,-0.953070,-0.584729,0.79,0,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,3
4,0.416144,0.844119,0.97,1,1,1,0,0,0,0,...,1,0,0,0,1,0,0,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.175190,-0.494551,0.79,0,1,0,0,0,0,1,...,0,1,0,1,0,0,0,1,0,10
996,-0.508853,0.633952,0.98,1,1,0,0,1,0,0,...,1,0,0,0,0,1,0,0,1,5
997,0.674380,-0.598325,0.09,1,1,0,0,0,0,1,...,0,0,1,0,1,0,0,0,1,4
998,0.862906,-0.374151,0.57,1,1,0,0,1,0,0,...,0,1,0,0,1,0,0,1,0,8


In [124]:
# #Split the data into test and training sets using pandas
# msk = np.random.rand(len(df_normalized)) < 0.8
# train = df_normalized[msk]
# test = df_normalized[~msk]


In [5]:
#Turn the pandas dataframe into a numpy array, split out the label and then break into sets
import tensorflow as tf

#turn df_normalized into a normal array
data=df_normalized.values

#Get all columns except the output column
X=data[:,1:]
#Get the output column
y=data[:,-1]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [87]:
X_train.shape[1]

34

In [6]:
#Time to build the model

#Simple feed-forward model. No U-nets or anything fancy
ann = tf.keras.models.Sequential()

neurons=1000

#Input and first hidden layer
ann.add(tf.keras.layers.Dense(units=neurons, activation='relu'))
ann.add(tf.keras.layers.Dense(units=neurons, activation='relu'))
ann.add(tf.keras.layers.Dense(units=neurons, activation='relu'))

#Output layer
#Classifier head
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

#Compile!
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])



In [7]:
#Train the MODEL!!!!!
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
25/25 [==============================] - 1s 16ms/step - loss: -7628.2061 - accuracy: 0.0675
Epoch 2/100
25/25 [==============================] - 0s 17ms/step - loss: -229403.0469 - accuracy: 0.0662
Epoch 3/100
25/25 [==============================] - 0s 17ms/step - loss: -2106301.5000 - accuracy: 0.0662
Epoch 4/100
25/25 [==============================] - 0s 16ms/step - loss: -11104518.0000 - accuracy: 0.0662
Epoch 5/100
25/25 [==============================] - 0s 16ms/step - loss: -39009288.0000 - accuracy: 0.0662
Epoch 6/100
25/25 [==============================] - 0s 16ms/step - loss: -107899576.0000 - accuracy: 0.0662
Epoch 7/100
25/25 [==============================] - 0s 16ms/step - loss: -249086784.0000 - accuracy: 0.0662
Epoch 8/100
25/25 [==============================] - 0s 16ms/step - loss: -502267488.0000 - accuracy: 0.0662
Epoch 9/100
25/25 [==============================] - 0s 17ms/step - loss: -913465984.0000 - accuracy: 0.0662
Epoch 10/100
25/25 [===========